# pytorch 的自动求导
## requires_grad
requires_grad: 一个张量的属性，用于指定是否需要计算该张量的梯度。如果requires_grad被设置为True，则表示需要计算该张量的梯度；如果设置为False，则表示不需要计算该张量的梯度。

变量的requires_grad属性默认为False，如果某一个节点requires_grad被设置为True，那么所有有它产生的张量requires_grad都是True
## grad_fn
grad_fn: 是一个指向创建该张量的函数的引用,指向创建该张量的函数，如加法、乘法、卷积等。通过追踪计算图中的依赖关系，。grad_fn记录了张量是如何被计算出来的，因此它可以用于构建计算图和进行自动求导。
# grad
grad：是一个属性（attribute），用于获取张量的梯度。它存储了相对于该张量的梯度值。

# 示例
a和b的requires_grad属性都被设置为True，表示需要计算它们的梯度。然后，通过加法操作生成了新的张量c，并使用torch.sum()计算了d。这些张量的requires_grad属性都是True，并且它们的grad_fn属性分别指向对应的计算操作。

requires_grad和grad_fn是PyTorch中自动求导机制的关键概念。requires_grad属性用于指定是否需要计算梯度，而grad_fn属性用于建立计算图和追踪计算历史。它们共同作用，使得PyTorch能够自动计算梯度和进行反向传播。

In [1]:
import torch

a = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
b = torch.tensor([4.0, 5.0, 6.0], requires_grad=True)

c = a + b
d = torch.sum(c)

print(c.requires_grad)  # True
print(c.grad_fn)  # <AddBackward0 object at 0x7f7e8f272fd0>

print(d.requires_grad)  # True
print(d.grad_fn)  # <SumBackward0 object at 0x7f7e8f272fd0>


True
True


# backward()方法
在PyTorch中，backward()函数用于计算张量的梯度。它的参数是可选的，但是，在大多数情况下，可以省略backward()方法的参数，使用默认的权重为1来计算梯度。
* gradient（梯度）：用于指定计算梯度的权重。默认情况下，gradient参数为None，表示使用张量的值自动计算梯度。如果想自定义梯度的权重，可以传递一个与张量具有相同形状的张量作为gradient参数。

* retain_graph（保留计算图）：一个布尔值，用于指定是否保留计算图以供后续计算梯度。默认情况下，retain_graph参数为False，表示在调用backward()方法后会释放计算图。如果需要多次调用backward()方法来计算不同的梯度，可以将retain_graph设置为True。

* create_graph（创建计算图）：一个布尔值，用于指定是否创建一个新的计算图以供计算二阶导数。默认情况下，create_graph参数为False，表示不会创建二阶导数的计算图。如果需要计算二阶导数，可以将create_graph设置为True。

In [ ]:
# torch.ones(c.size()) * 2创建了一个与c形状相同的张量，其中的元素值都为2。这个张量作为参数传递给backward()方法，表示将梯度乘以2进行加权处理。
c.backward(torch.ones(c.size()) * 2)


* torch.no_grad()是一个上下文管理器，用于指定在其作用域内的操作不需要计算梯度。在深度学习中，有时候我们只需要进行前向传播，而不需要计算梯度，例如在测试阶段或者只需要得到模型的输出结果。
      import torch

      x = torch.randn(5, 5, requires_grad=True)
      y = torch.randn(5, 5)

      with torch.no_grad():
          z = x + y  # 不会计算z的梯度

      z.backward()  # 由于z处于no_grad上下文中，不会计算任何梯度
z的计算不会被跟踪，因为它处于torch.no_grad()的上下文中。

* torch.set_grad_enabled()是一个函数，用于临时禁用或启用梯度计算。当梯度计算被禁用时，PyTorch将不会跟踪张量的梯度，从而节省内存和计算资源。
      import torch

      x = torch.randn(5, 5, requires_grad=True)
      y = torch.randn(5, 5)

      torch.set_grad_enabled(False)

      z = x + y  # 不会计算z的梯度

      z.backward()  # 由于梯度计算被禁用，不会计算任何梯度

      torch.set_grad_enabled(True)  # 再次启用梯度计算

      z = x + y  # 重新计算z

      z.backward()  # 计算x的梯度
调用torch.set_grad_enabled(False)将梯度计算禁用。在这个上下文中，计算z不会跟踪梯度，因此调用z.backward()时不会计算任何梯度。然后，通过调用torch.set_grad_enabled(True)重新启用梯度计算，并重新计算z的值和梯度


# autograd的扩展——为自己写的函数写微分
    from torch.autograd import Function
    class MultiplyAdd(Function):
                                                            
        @staticmethod
        def forward(ctx, w, x, b):                              
            ctx.save_for_backward(w,x)
            output = w * x + b
            return output
        
        @staticmethod
        def backward(ctx, grad_output):                         
            w,x = ctx.saved_tensors
            grad_w = grad_output * x
            grad_x = grad_output * w
            grad_b = grad_output * 1
            return grad_w, grad_x, grad_b     
* 在PyTorch的autograd引擎中，ctx（也可以是其他任意的变量名）是用来保存中间计算结果和状态的上下文对象。

* MultiplyAdd有两个静态方法：forward和backward。forward方法接收三个输入：w、x和b。它使用ctx.save_for_backward方法将w和x保存在上下文中。然后，它通过将w和x相乘并加上b来计算输出。最后，它返回输出。

* backward方法接收一个输入：grad_output，表示输出相对于某个标量值的梯度。它使用上下文中保存的张量w和x来计算w、x和b的梯度。它将grad_output乘以x得到w的梯度，将grad_output乘以w得到x的梯度，将grad_output乘以1得到b的梯度。最后，它返回这些梯度。

这个自定义函数可以在PyTorch模型中使用，用于执行乘加操作并在反向传播过程中计算梯度。

在PyTorch中，.apply()是用于调用自定义函数的方法。它允许您将自定义函数应用于张量或变量。因此，调用这个类的这些方法，需要：

    x = t.ones(1)
    w = t.rand(1, requires_grad = True)
    b = t.rand(1, requires_grad = True)
    # 开始前向传播
    z=MultiplyAdd.apply(w, x, b)
    # 开始反向传播
    z.backward()
更多内容，

例如获取中间计算梯度，hook方法

高阶求导

查询官方文档